## IGS Report (Correlation between air pollution & socialeconomic factors)

## Structure
Introduction (10%)

Literature Review (20%-25%)

Data & Summary Statistic (5%-10%)

1. Geographic Boundary
   - All data is in 2011 lsoa boundary
2. Air Quality Data
   - 2006, 2008, 2010, 2013, 2016, 2019 (LSOA level)

Empirical Design (10%)

1. Ordinary Least Squares Regression Model (OLS)
2. Spatial Lag Model (SAR）
3. Global Moran's I
4. Spatial Durbin Panel Model ()

Empitical Results (20%)

Discussion (20%)

Conclusion (10%)

## Air Pollution Data in 2010 and 2008 (uncleaned)

### Get Geography

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.colors as mpc

import numpy as np

import os

import rasterio as rio
import rasterio.mask
from rasterio.plot import show
from rasterstats import zonal_stats

In [2]:
census_path = Path("../data/Geographic Boundary in 2011 and 2021/")
lsoa11 = gpd.read_file(census_path/"london-2011-lsoa.shp")
lsoa11 = lsoa11.set_index(lsoa11.LSOA11CD)

lsoa21 = gpd.read_file(census_path/"london-2021-lsoa-gen20.shp")
lsoa21 = lsoa21.set_index(lsoa21.LSOA21CD)

msoa21 = gpd.read_file(census_path/"london-2021-msoa-gen20.shp")
msoa21 = msoa21.set_index(msoa21.MSOA21CD)

lad21 = gpd.read_file(census_path/"london-2021-lad22-gen20.shp")
lad21 = lad21.set_index(lad21.LAD22CD)

In [3]:
from datetime import date
print(f'Last tested: {date.today()}')

Last tested: 2024-04-07


In [4]:
aq_opath = Path("../data/London Air Quality/")

In [5]:
def summarise_aq_raster(london_geo, rio_dr, label):
    print("Calculating zonal stats for " + label)
    zs_dict = zonal_stats(london_geo, rio_dr.read(1),    #input 2d raster (dataset reader is 3D)
                           affine=rio_dr.meta['transform'],
                           nodata=rio_dr.meta['nodata'],
                           stats=['min', 'max', 'mean', 'median', 'std'],
                           prefix=label)
    zs_pd = pd.DataFrame(zs_dict)
    zs_pd.set_index(london_geo.index,inplace=True)
    zs_pd=zs_pd.round(3)
    return zs_pd

In [6]:
#dict to use in loops below  (label:filename)
dict_2010 = {
    "PM25":"pm25_2010.asc",
    "PM10":"pm10mean_2010.asc",
    "PM10d":"pm10daily_2010.asc",
    "NOx":"nox_2010.asc",
    "NO2":"no2_2010.asc"
}

In [7]:
for k, v in dict_2010.items():
    rio_dr_2010 = rio.open(aq_opath/"LAEI2010-Concentrations-Data-ASCII"/v)
    pd2010 = summarise_aq_raster(lsoa11, rio_dr_2010, "{0}-2010-".format(k))
    pd2010.to_csv(aq_opath/"lsoa11-2010-{0}.csv".format(k))

Calculating zonal stats for PM25-2010-
Calculating zonal stats for PM10-2010-
Calculating zonal stats for PM10d-2010-
Calculating zonal stats for NOx-2010-
Calculating zonal stats for NO2-2010-


In [8]:
#dict to use in loops below  (label:filename)
dict_2008 = {
    "PM25":"pm25_2008.asc",
    "PM10":"pm10mean_2008.asc",
    "PM10d":"pm10daily_2008.asc",
    "NOx":"nox_2008.asc",
    "NO2":"no2_2008.asc"
}

In [9]:
for k, v in dict_2008.items():
    rio_dr_2008 = rio.open(aq_opath/"LAEI2010-Concentrations-Data-ASCII"/v)
    pd2008 = summarise_aq_raster(lsoa11, rio_dr_2008, "{0}-2008-".format(k))
    pd2008.to_csv(aq_opath/"lsoa11-2008-{0}.csv".format(k))

Calculating zonal stats for PM25-2008-
Calculating zonal stats for PM10-2008-
Calculating zonal stats for PM10d-2008-
Calculating zonal stats for NOx-2008-
Calculating zonal stats for NO2-2008-


In [10]:
# Define the files of old names to new names
file_mapping = {
    'lsoa11-2008-NO2.csv': 'london-2008-NO2.csv',
    'lsoa11-2008-NOx.csv': 'london-2008-NOx.csv',
    'lsoa11-2008-PM10.csv': 'london-2008-PM10.csv',
    'lsoa11-2008-PM10d.csv': 'london-2008-PM10d.csv',
    'lsoa11-2008-PM25.csv':'london-2008-PM25.csv',
    'lsoa11-2010-NO2.csv':'london-2010-NO2.csv',
    'lsoa11-2010-NOx.csv':'london-2010-NOx.csv',
    'lsoa11-2010-PM10.csv':'london-2010-PM10.csv',
    'lsoa11-2010-PM10d.csv':'london-2010-PM10d.csv',
    'lsoa11-2010-PM25.csv':'london-2010-PM25.csv'
}

for filename in os.listdir(aq_opath):
    if filename in file_mapping:
        # Construct the old and new file paths
        old_filepath = os.path.join(aq_opath, filename)
        new_filename = file_mapping[filename]
        new_filepath = os.path.join(aq_opath, new_filename)
        
        # Rename the file
        os.rename(old_filepath, new_filepath)
        print(f"Renamed '{filename}' to '{new_filename}'")

## Merge all air quality data together

In [11]:
import pandas as pd
from pathlib import Path
from stargazer.stargazer import Stargazer
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import os

In [12]:
aq2008no2 = pd.read_csv("../data/London Air Quality/london-2008-NO2.csv")
aq2008nox = pd.read_csv("../data/London Air Quality/london-2008-NOx.csv")
aq2008pm10 = pd.read_csv("../data/London Air Quality/london-2008-PM10.csv")
aq2008pm25 = pd.read_csv("../data/London Air Quality/london-2008-PM25.csv")
aq2010no2 = pd.read_csv("../data/London Air Quality/london-2010-NO2.csv")
aq2010nox = pd.read_csv("../data/London Air Quality/london-2010-NOx.csv")
aq2010pm10 = pd.read_csv("../data/London Air Quality/london-2010-PM10.csv")
aq2010pm25 = pd.read_csv("../data/London Air Quality/london-2010-PM25.csv")
aq2013no2 = pd.read_csv("../data/London Air Quality/london-2013-NO2.csv")
aq2013nox = pd.read_csv("../data/London Air Quality/london-2013-NOx.csv")
aq2013pm10 = pd.read_csv("../data/London Air Quality/london-2013-PM10.csv")
aq2013pm25 = pd.read_csv("../data/London Air Quality/london-2013-PM25.csv")
aq2016no2 = pd.read_csv("../data/London Air Quality/london-2016-NO2.csv")
aq2016nox = pd.read_csv("../data/London Air Quality/london-2016-NOx.csv")
aq2016pm10 = pd.read_csv("../data/London Air Quality/london-2016-PM10.csv")
aq2016pm25 = pd.read_csv("../data/London Air Quality/london-2016-PM25.csv")
aq2019no2 = pd.read_csv("../data/London Air Quality/london-2019-NO2.csv")
aq2019nox = pd.read_csv("../data/London Air Quality/london-2019-NOx.csv")
aq2019pm10 = pd.read_csv("../data/London Air Quality/london-2019-PM10.csv")
aq2019pm25 = pd.read_csv("../data/London Air Quality/london-2019-PM25.csv")

In [13]:
aq2019pm10

,LSOA11CD,PM10-2019-min,PM10-2019-max,PM10-2019-mean,PM10-2019-std,PM10-2019-median
0,E01000001,21.401,43.266,23.851,4.364,22.336
1,E01000002,21.555,42.033,22.713,2.563,22.205
2,E01000003,20.941,48.336,21.892,2.803,21.452
3,E01000005,20.569,61.689,23.911,4.951,22.422
4,E01000006,15.846,18.664,16.259,0.260,16.222
...,...,...,...,...,...,...
4830,E01033742,15.617,16.375,15.741,0.138,15.695
4831,E01033743,14.967,26.849,15.378,0.698,15.247
4832,E01033744,15.697,24.278,16.223,0.756,16.058
4833,E01033745,15.067,31.394,16.199,1.813,15.621


In [14]:
dfs = [aq2008no2,aq2008nox,aq2008pm10,aq2008pm25,aq2010no2,aq2010nox,aq2010pm10,aq2010pm25,aq2013no2,aq2013nox,aq2013pm10,aq2013pm25,aq2016no2,aq2016nox,aq2016pm10,aq2016pm25,aq2019no2,aq2019nox,aq2019pm10,aq2019pm25]
df2008 = dfs[0]
df2010 = dfs[4]
df2013 = dfs[8]
df2016 = dfs[12]
df2019 = dfs[16]

In [15]:
for df in dfs[1:4]:
    df2008 = pd.merge(df2008, df, left_on='LSOA11CD', right_on='LSOA11CD')

for df in dfs[5:8]:
    df2010 = pd.merge(df2010, df, left_on='LSOA11CD', right_on='LSOA11CD')

for df in dfs[9:12]:
    df2013 = pd.merge(df2013, df, left_on='LSOA11CD', right_on='LSOA11CD')

for df in dfs[13:16]:
    df2016 = pd.merge(df2016, df, left_on='LSOA11CD', right_on='LSOA11CD')

for df in dfs[17:]:
    df2019 = pd.merge(df2019, df, left_on='LSOA11CD', right_on='LSOA11CD')

In [16]:
df2008.columns

Index(['LSOA11CD', 'NO2-2008-min', 'NO2-2008-max', 'NO2-2008-mean',
       'NO2-2008-std', 'NO2-2008-median', 'NOx-2008-min', 'NOx-2008-max',
       'NOx-2008-mean', 'NOx-2008-std', 'NOx-2008-median', 'PM10-2008-min',
       'PM10-2008-max', 'PM10-2008-mean', 'PM10-2008-std', 'PM10-2008-median',
       'PM25-2008-min', 'PM25-2008-max', 'PM25-2008-mean', 'PM25-2008-std',
       'PM25-2008-median'],
      dtype='object')

In [17]:
df2010.columns

Index(['LSOA11CD', 'NO2-2010-min', 'NO2-2010-max', 'NO2-2010-mean',
       'NO2-2010-std', 'NO2-2010-median', 'NOx-2010-min', 'NOx-2010-max',
       'NOx-2010-mean', 'NOx-2010-std', 'NOx-2010-median', 'PM10-2010-min',
       'PM10-2010-max', 'PM10-2010-mean', 'PM10-2010-std', 'PM10-2010-median',
       'PM25-2010-min', 'PM25-2010-max', 'PM25-2010-mean', 'PM25-2010-std',
       'PM25-2010-median'],
      dtype='object')

In [18]:
df2013.columns

Index(['LSOA11CD', 'NO2-2013-min', 'NO2-2013-max', 'NO2-2013-mean',
       'NO2-2013-std', 'NO2-2013-median', 'NOx-2013-min', 'NOx-2013-max',
       'NOx-2013-mean', 'NOx-2013-std', 'NOx-2013-median', 'PM10-2013-min',
       'PM10-2013-max', 'PM10-2013-mean', 'PM10-2013-std', 'PM10-2013-median',
       'PM25-2013-min', 'PM25-2013-max', 'PM25-2013-mean', 'PM25-2013-std',
       'PM25-2013-median'],
      dtype='object')

In [19]:
df2016.columns

Index(['LSOA11CD', 'NO2-2016-min', 'NO2-2016-max', 'NO2-2016-mean',
       'NO2-2016-std', 'NO2-2016-median', 'NOx-2016-min', 'NOx-2016-max',
       'NOx-2016-mean', 'NOx-2016-std', 'NOx-2016-median', 'PM10-2016-min',
       'PM10-2016-max', 'PM10-2016-mean', 'PM10-2016-std', 'PM10-2016-median',
       'PM25-2016-min', 'PM25-2016-max', 'PM25-2016-mean', 'PM25-2016-std',
       'PM25-2016-median'],
      dtype='object')

In [20]:
df2019.columns

Index(['LSOA11CD', 'NO2-2019-min', 'NO2-2019-max', 'NO2-2019-mean',
       'NO2-2019-std', 'NO2-2019-median', 'NOx-2019-min', 'NOx-2019-max',
       'NOx-2019-mean', 'NOx-2019-std', 'NOx-2019-median', 'PM10-2019-min',
       'PM10-2019-max', 'PM10-2019-mean', 'PM10-2019-std', 'PM10-2019-median',
       'PM25-2019-min', 'PM25-2019-max', 'PM25-2019-mean', 'PM25-2019-std',
       'PM25-2019-median'],
      dtype='object')

In [21]:
#merge all air quality data
all_data = pd.merge(df2008,
                    df2010,
                   left_on ='LSOA11CD',
                   right_on='LSOA11CD',
                   how='inner')

all_data = pd.merge(all_data,
                    df2013,
                   left_on ='LSOA11CD',
                   right_on='LSOA11CD',
                   how='inner')

all_data = pd.merge(all_data,
                    df2016,
                   left_on ='LSOA11CD',
                   right_on='LSOA11CD',
                   how='inner')

all_data = pd.merge(all_data,
                    df2019,
                   left_on ='LSOA11CD',
                   right_on='LSOA11CD',
                   how='inner')
all_data

,LSOA11CD,NO2-2008-min,NO2-2008-max,NO2-2008-mean,NO2-2008-std,NO2-2008-median,NOx-2008-min,NOx-2008-max,NOx-2008-mean,NOx-2008-std,...,PM10-2019-min,PM10-2019-max,PM10-2019-mean,PM10-2019-std,PM10-2019-median,PM25-2019-min,PM25-2019-max,PM25-2019-mean,PM25-2019-std,PM25-2019-median
0,E01000001,53.194,121.742,65.384,12.199,60.720,103.217,299.872,134.977,33.468,...,21.401,43.266,23.851,4.364,22.336,12.721,20.290,13.563,1.596,13.002
1,E01000002,52.883,82.607,57.591,4.745,56.627,102.253,178.725,115.046,12.250,...,21.555,42.033,22.713,2.563,22.205,12.688,20.162,13.065,0.920,12.879
2,E01000003,51.619,82.854,55.053,4.461,53.709,98.173,179.355,107.362,11.987,...,20.941,48.336,21.892,2.803,21.452,12.558,21.677,12.877,0.937,12.722
3,E01000005,63.255,161.386,75.521,15.369,68.698,135.551,429.098,169.800,44.419,...,20.569,61.689,23.911,4.951,22.422,12.406,22.884,13.392,1.416,12.963
4,E01000006,34.442,38.164,35.605,0.706,35.495,53.734,65.454,57.125,2.103,...,15.846,18.664,16.259,0.260,16.222,10.431,11.604,10.606,0.101,10.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4830,E01033742,36.404,38.659,36.984,0.490,36.833,57.315,63.550,59.103,1.463,...,15.617,16.375,15.741,0.138,15.695,10.232,10.510,10.285,0.054,10.268
4831,E01033743,31.964,49.961,33.849,1.465,33.649,47.394,94.164,51.727,3.723,...,14.967,26.849,15.378,0.698,15.247,10.041,14.373,10.177,0.244,10.135
4832,E01033744,36.455,57.493,37.692,1.768,37.458,57.480,117.913,61.234,5.023,...,15.697,24.278,16.223,0.756,16.058,10.266,13.161,10.468,0.259,10.416
4833,E01033745,32.357,59.797,35.869,3.483,35.219,48.432,129.447,57.321,9.596,...,15.067,31.394,16.199,1.813,15.621,10.065,15.630,10.448,0.606,10.260


## Merge air quality data with geographic information

In [22]:
geometry = pd.read_csv('../data/Merged_LSOA_Data.csv')

In [23]:
geometry

,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,geometry,USUALRES,...,PM10max,PM10sd,NOxmean,NOxmin,NOxmax,NOxsd,NO2mean,NO2min,NO2max,NO2sd
0,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London,"POLYGON ((532105.0919998939 182011.23, 532162....",1465,...,40.9147,2.5343,116.7335,74.3843,368.4532,46.8249,57.8972,43.0877,127.2501,14.9486
1,E01000002,City of London 001B,E02000001,City of London 001,E09000001,City of London,E12000007,London,"POLYGON ((532746.8130000263 181786.891000028, ...",1436,...,32.2687,0.9467,86.3397,72.6057,191.7217,17.0302,47.6356,42.3820,80.7450,6.3878
2,E01000003,City of London 001C,E02000001,City of London 001,E09000001,City of London,E12000007,London,"POLYGON ((532135.1449999654 182198.1190000199,...",1346,...,33.6703,1.0226,86.1607,74.2665,189.0205,17.5927,47.3078,42.9148,81.4941,6.0950
3,E01000005,City of London 001E,E02000001,City of London 001,E09000001,City of London,E12000007,London,"POLYGON ((533807.9460001207 180767.7700000888,...",985,...,59.8320,3.8027,132.1101,76.5674,893.0905,89.5282,61.1162,43.7231,234.1228,23.2659
4,E01000006,Barking and Dagenham 016A,E02000017,Barking and Dagenham 016,E09000002,Barking and Dagenham,E12000007,London,"POLYGON ((545122.048999952 184314.931000118, 5...",1703,...,25.0226,0.1474,51.3147,46.9127,57.4380,2.1936,32.5846,31.1267,34.9157,0.7742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4830,E01033742,Greenwich 007F,E02000319,Greenwich 007,E09000011,Greenwich,E12000007,London,"POLYGON ((544642.6800000862 179824.6740001431,...",1352,...,24.6248,0.1205,46.8990,44.4535,54.9514,2.2943,31.2253,30.4867,34.0319,0.7043
4831,E01033743,Greenwich 002H,E02000314,Greenwich 002,E09000011,Greenwich,E12000007,London,"POLYGON ((546579.1949997952 181097.8129996927,...",2038,...,27.7521,0.3605,43.6637,41.0214,98.0401,5.0553,29.8225,28.8556,48.8936,1.8149
4832,E01033744,Greenwich 007G,E02000319,Greenwich 007,E09000011,Greenwich,E12000007,London,"POLYGON ((544536.4859999884 179447.1149999507,...",1845,...,32.0366,0.7820,53.1747,46.1718,146.0446,9.9603,33.3076,31.0721,63.5436,3.3280
4833,E01033745,Greenwich 002I,E02000314,Greenwich 002,E09000011,Greenwich,E12000007,London,"POLYGON ((546415.7449998577 180152.2700002448,...",1820,...,32.4758,0.9603,50.4032,42.3186,140.0508,11.6730,32.2175,29.1870,63.2214,4.2280


In [24]:
geometry = geometry[['LSOA11CD','LSOA11NM','MSOA11CD','MSOA11NM','LAD11CD','LAD11NM','RGN11CD','RGN11NM','Area','geometry']]
geometry

,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,Area,geometry
0,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London,13.33,"POLYGON ((532105.0919998939 182011.23, 532162...."
1,E01000002,City of London 001B,E02000001,City of London 001,E09000001,City of London,E12000007,London,22.62,"POLYGON ((532746.8130000263 181786.891000028, ..."
2,E01000003,City of London 001C,E02000001,City of London 001,E09000001,City of London,E12000007,London,5.73,"POLYGON ((532135.1449999654 182198.1190000199,..."
3,E01000005,City of London 001E,E02000001,City of London 001,E09000001,City of London,E12000007,London,19.07,"POLYGON ((533807.9460001207 180767.7700000888,..."
4,E01000006,Barking and Dagenham 016A,E02000017,Barking and Dagenham 016,E09000002,Barking and Dagenham,E12000007,London,14.42,"POLYGON ((545122.048999952 184314.931000118, 5..."
...,...,...,...,...,...,...,...,...,...,...
4830,E01033742,Greenwich 007F,E02000319,Greenwich 007,E09000011,Greenwich,E12000007,London,12.33,"POLYGON ((544642.6800000862 179824.6740001431,..."
4831,E01033743,Greenwich 002H,E02000314,Greenwich 002,E09000011,Greenwich,E12000007,London,100.49,"POLYGON ((546579.1949997952 181097.8129996927,..."
4832,E01033744,Greenwich 007G,E02000319,Greenwich 007,E09000011,Greenwich,E12000007,London,15.23,"POLYGON ((544536.4859999884 179447.1149999507,..."
4833,E01033745,Greenwich 002I,E02000314,Greenwich 002,E09000011,Greenwich,E12000007,London,55.85,"POLYGON ((546415.7449998577 180152.2700002448,..."


In [25]:
all_data = pd.merge(geometry,
                    all_data,
                   left_on ='LSOA11CD',
                   right_on='LSOA11CD',
                   how='inner')

all_data

,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,Area,geometry,...,PM10-2019-min,PM10-2019-max,PM10-2019-mean,PM10-2019-std,PM10-2019-median,PM25-2019-min,PM25-2019-max,PM25-2019-mean,PM25-2019-std,PM25-2019-median
0,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London,13.33,"POLYGON ((532105.0919998939 182011.23, 532162....",...,21.401,43.266,23.851,4.364,22.336,12.721,20.290,13.563,1.596,13.002
1,E01000002,City of London 001B,E02000001,City of London 001,E09000001,City of London,E12000007,London,22.62,"POLYGON ((532746.8130000263 181786.891000028, ...",...,21.555,42.033,22.713,2.563,22.205,12.688,20.162,13.065,0.920,12.879
2,E01000003,City of London 001C,E02000001,City of London 001,E09000001,City of London,E12000007,London,5.73,"POLYGON ((532135.1449999654 182198.1190000199,...",...,20.941,48.336,21.892,2.803,21.452,12.558,21.677,12.877,0.937,12.722
3,E01000005,City of London 001E,E02000001,City of London 001,E09000001,City of London,E12000007,London,19.07,"POLYGON ((533807.9460001207 180767.7700000888,...",...,20.569,61.689,23.911,4.951,22.422,12.406,22.884,13.392,1.416,12.963
4,E01000006,Barking and Dagenham 016A,E02000017,Barking and Dagenham 016,E09000002,Barking and Dagenham,E12000007,London,14.42,"POLYGON ((545122.048999952 184314.931000118, 5...",...,15.846,18.664,16.259,0.260,16.222,10.431,11.604,10.606,0.101,10.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4830,E01033742,Greenwich 007F,E02000319,Greenwich 007,E09000011,Greenwich,E12000007,London,12.33,"POLYGON ((544642.6800000862 179824.6740001431,...",...,15.617,16.375,15.741,0.138,15.695,10.232,10.510,10.285,0.054,10.268
4831,E01033743,Greenwich 002H,E02000314,Greenwich 002,E09000011,Greenwich,E12000007,London,100.49,"POLYGON ((546579.1949997952 181097.8129996927,...",...,14.967,26.849,15.378,0.698,15.247,10.041,14.373,10.177,0.244,10.135
4832,E01033744,Greenwich 007G,E02000319,Greenwich 007,E09000011,Greenwich,E12000007,London,15.23,"POLYGON ((544536.4859999884 179447.1149999507,...",...,15.697,24.278,16.223,0.756,16.058,10.266,13.161,10.468,0.259,10.416
4833,E01033745,Greenwich 002I,E02000314,Greenwich 002,E09000011,Greenwich,E12000007,London,55.85,"POLYGON ((546415.7449998577 180152.2700002448,...",...,15.067,31.394,16.199,1.813,15.621,10.065,15.630,10.448,0.606,10.260


In [26]:
all_data.to_csv('../data/london_air_quality_data.csv', index=False)